
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


# Installation

In [ ]:
%pip install redis-retrieval-optimizer

## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 

In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [1]:
import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

[I 2025-06-11 17:30:55,706] A new study created in memory with name: test


17:30:58 datasets INFO   PyTorch version 2.3.0 available.
17:30:58 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
17:30:58 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:31:00 root INFO   Recreating index...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

17:33:14 redisvl.index.index INFO   Index already exists, overwriting.
17:33:14 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
17:33:14 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:33:15 root INFO   Recreating index...
17:33:15 root INFO   Corpus size: 3633
17:33:20 root INFO   Data indexed total_indexing_time=4.664s, num_docs=3633
17:33:22 root INFO   Saving metrics for study: 8836d90b-0ac1-44c5-a1ee-56d70b6facc0, METRICS={'search_method': ['vector', 'hybrid', 'hybrid', 'vector', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'vector', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'hybrid', 'vector', 'hybrid'], 'total_indexing_time': [4.754, 2.381, 2.575, 3.269, 3.892, 2.314, 4.929, 5.246, 4.067, 3.053, 3.098, 5.118, 5.118, 5.118, 6.552, 4.024, 5.46, 5.46, 2.745, 4.664], 'avg_query_time': [0.0018532099000440662, 0.0030282289251085406, 0.0030207508488705286, 0.0007963652950322296, 0.004091318915872013, 0.0028466705936396456, 0.0040379382508457995, 0.004082830328690379, 0.0018487087344237526, 0.0014829369890431501, 0.004105854329679035, 0.004502101579317737, 0.004363842423855336, 0.0042323209922011055, 0.004151501522713771, 0.00399246

[I 2025-06-11 17:33:22,338] Trial 19 finished with value: 4.664 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'hybrid', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 9, 'ef_runtime': 30, 'ef_construction': 200, 'm': 64}. Best is trial 14 with value: 6.552.


17:33:22 root INFO   Completed Bayesian optimization... 


17:33:22 root INFO   Best Configuration: 14: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'hybrid', 'algorithm': 'hnsw', 'var_dtype': 'float16', 'distance_metric': 'cosine', 'ret_k': 10, 'ef_runtime': 50, 'ef_construction': 250, 'm': 64}:


17:33:22 root INFO   Best Score: [6.552]




In [2]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "total_indexing_time", "f1@k"]].sort_values(by="f1@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,total_indexing_time,f1@k
10,hybrid,hnsw,float16,100,30,16,0.004106,3.098,0.130712
1,hybrid,hnsw,float32,150,30,64,0.003028,2.381,0.130712
17,hybrid,hnsw,float16,200,50,64,0.004096,5.460,0.130712
16,hybrid,hnsw,float16,200,50,64,0.003857,5.460,0.130712
15,hybrid,hnsw,float16,100,50,64,0.003992,4.024,0.130712
14,hybrid,hnsw,float16,250,50,64,0.004152,6.552,0.130712
13,hybrid,hnsw,float16,250,10,16,0.004232,5.118,0.130712
12,hybrid,hnsw,float16,250,10,16,0.004364,5.118,0.130712
11,hybrid,hnsw,float16,250,10,16,0.004502,5.118,0.130712
19,hybrid,hnsw,float16,200,30,64,0.003883,4.664,0.130712
